In [7]:
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict

In [32]:
datas = list(Path().glob("*.txt"))
datas

[WindowsPath('cifar10-kar-rcd.txt'),
 WindowsPath('cifar10-roar-plain.txt'),
 WindowsPath('cifar10-roar-rcd-fgm.txt'),
 WindowsPath('cifar10-roar-rcd-noabs.txt'),
 WindowsPath('cifar10-roar-rcd.txt'),
 WindowsPath('mnist-kar.txt'),
 WindowsPath('mnist-roar.txt')]

In [33]:
def read_file(record_path, no_attention=False):
    with record_path.open() as f:
        x = f.read().splitlines()[2:]
        if no_attention:
            x = [line for line in x if not (("resnetcbam" in line) or ("resnetanr" in line))]
        x = [line.strip("|").split("|") for line in x]
        m_type, a_type, percent, score = list(zip(*x))
        s_dict = defaultdict(list)
        for i in range(len(x)):
            s_dict[f"{m_type[i]}-{a_type[i]}"].append(np.float(score[i].strip("%")))

    percentages = np.arange(0, 100, 10).astype(np.float16)
    return s_dict

def read_all(datas):
    df_dict = defaultdict()
    for d in datas:
        k = d.name.rstrip(".txt")
        idx = np.arange(0, 100, 10)
        v = pd.DataFrame(read_file(d), index=idx)
        df_dict[k] = v
    return df_dict

In [34]:
df_dict = read_all(datas)
df_dict.keys()

dict_keys(['cifar10-kar-rcd', 'cifar10-roar-plain', 'cifar10-roar-rcd-fgm', 'cifar10-roar-rcd-noabs', 'cifar10-roar-rcd', 'mnist-kar', 'mnist-roar'])

In [45]:
# experiment 1
exp1 = df_dict['cifar10-roar-rcd-noabs'] - df_dict['cifar10-roar-rcd']
exp1

,resnet-random,resnet-vanillagrad,resnet-inputgrad,resnet-guidedgrad,resnet-gradcam,resnetcbam-random,resnetcbam-vanillagrad,resnetcbam-inputgrad,resnetcbam-guidedgrad,resnetcbam-gradcam,resnetanr-random,resnetanr-vanillagrad,resnetanr-inputgrad,resnetanr-guidedgrad,resnetanr-gradcam
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10,-0.22,15.02,14.27,1.31,-0.42,-0.13,6.40,9.20,0.51,-0.06,-0.26,15.19,14.84,5.53,0.42
20,0.59,20.99,19.03,5.11,0.05,-1.12,14.24,12.60,2.87,-0.51,-0.99,21.98,19.14,9.93,-1.17
30,1.33,24.53,21.47,8.10,0.68,0.50,15.28,12.69,5.89,0.19,0.01,25.71,20.75,13.84,-0.21
40,2.81,26.21,22.45,9.75,-0.78,0.40,14.90,13.43,9.14,-0.64,-0.31,29.35,21.32,16.43,-0.17
50,3.78,30.02,23.17,9.33,0.14,0.01,17.61,11.78,11.53,0.80,-0.68,30.54,21.73,19.67,-1.12
60,4.56,32.90,25.16,13.44,-0.34,3.28,20.12,11.59,13.31,3.12,1.15,32.84,24.64,25.57,0.05
70,6.09,34.68,26.42,17.90,0.58,2.14,18.83,8.88,17.84,1.78,0.69,30.40,22.11,27.23,-0.21
80,3.34,33.14,25.31,22.85,1.08,4.30,20.44,9.81,17.36,0.84,-0.93,32.67,21.04,30.56,-1.32
90,4.42,28.15,21.74,19.19,0.41,5.70,19.62,8.01,22.18,2.06,-1.30,31.26,19.95,30.99,0.88


In [36]:
# experiment 2
exp2 = df_dict['cifar10-roar-plain'] - df_dict['cifar10-roar-rcd']
exp2

,resnet-random,resnet-vanillagrad,resnet-inputgrad,resnet-guidedgrad,resnet-gradcam,resnetcbam-random,resnetcbam-vanillagrad,resnetcbam-inputgrad,resnetcbam-guidedgrad,resnetcbam-gradcam,resnetanr-random,resnetanr-vanillagrad,resnetanr-inputgrad,resnetanr-guidedgrad,resnetanr-gradcam
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10,-0.27,-0.53,0.03,-0.02,-0.80,0.60,-0.62,0.57,0.95,0.67,-0.86,-1.00,0.47,0.39,0.39
20,1.37,-1.07,-0.18,-0.56,1.55,-0.72,2.16,1.66,-0.25,0.43,-0.15,-0.41,0.33,-0.37,-0.60
30,-0.18,0.46,-1.14,1.85,1.91,-0.30,0.94,0.38,-0.38,-1.35,0.78,0.58,-0.33,1.50,-0.66
40,0.65,-0.54,-0.77,1.57,-1.21,-0.24,-0.03,0.56,0.40,-0.98,0.74,0.60,0.34,1.07,0.82
50,-0.36,0.13,-0.77,-1.97,1.07,-2.10,1.39,0.81,-0.08,-0.57,1.01,1.29,-0.67,-0.54,0.03
60,0.36,-1.76,-0.11,-3.09,-1.28,-0.55,1.24,0.69,0.12,1.29,1.08,0.19,-1.38,-0.53,-0.69
70,0.63,0.12,-0.51,1.02,-0.84,-0.36,-1.82,1.62,0.29,2.21,0.39,-0.73,0.08,0.84,-0.31
80,0.17,0.74,2.41,-0.21,0.30,-1.32,0.15,0.70,-1.61,1.27,1.13,-3.21,-0.94,0.96,-1.27
90,-2.08,-2.87,0.28,-2.60,-2.34,-0.81,1.39,-2.00,-0.66,-0.23,-2.44,2.19,0.22,1.81,1.55


In [50]:
exp2.agg(["mean", "std"], axis=1)

,mean,std
0,0.000000,0.000000
10,-0.002000,0.636982
20,0.212667,1.002685
30,0.270667,1.012052
40,0.198667,0.800757
50,-0.088667,1.082404
60,-0.294667,1.224219
70,0.175333,1.017138
80,-0.048667,1.412752
90,-0.572667,1.762942


In [46]:
# experiment 3
exp3 = df_dict['cifar10-roar-rcd-fgm'] - df_dict['cifar10-roar-rcd']
exp3

,resnet-random,resnet-vanillagrad,resnet-inputgrad,resnet-guidedgrad,resnet-gradcam,resnetcbam-random,resnetcbam-vanillagrad,resnetcbam-inputgrad,resnetcbam-guidedgrad,resnetcbam-gradcam,resnetanr-random,resnetanr-vanillagrad,resnetanr-inputgrad,resnetanr-guidedgrad,resnetanr-gradcam
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
10,5.44,1.58,1.64,-0.44,2.12,5.28,5.37,3.34,-3.17,2.51,5.04,5.04,3.07,-0.36,1.78
20,8.43,2.05,2.41,-1.59,2.59,7.03,7.66,4.06,-3.69,3.24,5.15,4.09,1.74,-0.14,1.67
30,9.26,4.28,2.25,-0.54,4.16,8.25,8.72,2.27,-2.92,1.66,6.00,5.84,1.71,0.67,1.69
40,9.57,2.77,0.92,0.93,2.16,9.33,5.25,2.16,-2.02,2.36,6.56,4.35,2.57,-0.18,2.42
50,9.97,5.05,1.11,-2.04,3.58,8.71,7.94,2.87,-0.35,4.81,5.94,4.27,1.13,0.88,2.26
60,8.41,3.30,2.16,-0.69,2.48,9.04,9.51,1.23,-0.48,4.89,5.60,5.41,2.05,1.77,2.85
70,9.81,3.95,0.45,-0.28,3.34,8.56,4.11,0.36,-1.75,4.61,3.23,2.04,3.83,2.42,3.24
80,6.34,2.90,0.92,-0.52,5.49,5.59,6.35,-0.31,-2.11,7.03,3.85,3.29,3.33,4.67,3.26
90,4.57,1.23,3.32,-1.90,6.76,4.37,4.18,2.28,1.63,3.68,-0.55,5.90,4.59,5.35,7.70


In [51]:
exp3.agg(["mean", "std"], axis=1).mean(0)

mean    3.001067
std     2.774183
dtype: float64